<a href="https://colab.research.google.com/github/KaviniA2003/iris-flower-classification/blob/main/Email_Tone_Rewriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Install necessary packages silently
!pip install transformers sentencepiece gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import gradio as gr

# Load T5 model and tokenizer (t5-base)
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

print("Model and tokenizer loaded successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [18]:
def rewrite_email_tone(email_text: str, tone: str) -> str:
    """
    Rewrites the input email text in the specified tone.

    Parameters:
    - email_text (str): The original email content.
    - tone (str): Desired tone ('friendly', 'formal', 'professional', 'casual')

    Returns:
    - str: Rewritten email text in the chosen tone.
    """

    # Clean input
    email_text = email_text.strip()
    tone = tone.lower().strip()

    # Prepare T5 prompt for tone paraphrasing
    tone_prompts = {
        "friendly": "paraphrase in a friendly tone: ",
        "formal": "paraphrase in a formal tone: ",
        "professional": "paraphrase in a professional tone: ",
        "casual": "paraphrase in a casual tone: "
    }

    prompt = tone_prompts.get(tone, "paraphrase: ") + email_text

    # Encode prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate paraphrase
    outputs = model.generate(input_ids,
                             max_length=512,
                             num_beams=5,
                             early_stopping=True,
                             no_repeat_ngram_size=2)

    # Decode output tokens
    rewritten_email = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return rewritten_email


In [19]:
sample_email = """
Dear team,

I would like to remind you about the upcoming project deadline next Friday. Please ensure all your deliverables are submitted on time.

Thank you for your cooperation.

Best regards,
Alex
"""

print("Original Email:")
print(sample_email)

print("\n--- Rewritten Samples ---\n")

for tone in ["friendly", "formal", "professional", "casual"]:
    print(f"Tone: {tone.capitalize()}")
    print(rewrite_email_tone(sample_email, tone))
    print("-" * 40)


Original Email:

Dear team,

I would like to remind you about the upcoming project deadline next Friday. Please ensure all your deliverables are submitted on time.

Thank you for your cooperation.

Best regards,
Alex


--- Rewritten Samples ---

Tone: Friendly
jimmy koenig: i would like to remind you about the upcoming project deadline next Friday . he says please ensure all your deliverables are submitted on time. best wishes, Alex.
----------------------------------------
Tone: Formal
cnn.com's ireport boot camp: i'd like to remind you about the upcoming project deadline next Friday . please ensure all your deliverables are submitted on time.
----------------------------------------
Tone: Professional
cnn's ireport: tell us about the upcoming project deadline next Friday . hello team, I would like to thank you for your cooperation.
----------------------------------------
Tone: Casual
cnn.com's ireport boot camp: i'd like to remind you about the upcoming project deadline next Friday 

In [20]:
def gradio_interface(email_text, tone):
    return rewrite_email_tone(email_text, tone)

demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(lines=10, label="Original Email", placeholder="Paste your email here..."),
        gr.Radio(choices=["friendly", "formal", "professional", "casual"], label="Select Tone")
    ],
    outputs=gr.Textbox(lines=10, label="Rewritten Email"),
    title="Smart Email Tone Rewriter",
    description="Rewrite your emails in different tones using an AI model."
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://059b3c2cc6ecdbff1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
